In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from PIL import Image

## Простой пример сверточной сети - AlexNet

Структуру в сети задаем через Sequential

[Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html)

**Сверточный слой**

3, 64, - количество входных и выходных каналов

kernel_size - высота и ширина сверточного фильтра (можно использовать скаляр или кортеж, например (3, 5)

stride - значение в пикселях, на которое смещается сверточный фильтр. Можно кортеж, если мы хотим двигаться не только в одном измерении

padding - отступ, которым мы дополняем кадр, если не хватаетпикселов для полной проходки фильтром

[Conv2d](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html)

**Пуллинг**

В нашем случае, макспулинг получаем максимальное значение для каждого из тензоров. Еще вариант - использовать среднее., а так-же адаптивное среднее и адаптивный максимум для данных, приводящих к разным входным тензорам.

[MaxPool2d](https://pytorch.org/docs/master/generated/torch.nn.MaxPool2d.html)

**Дропаут**
[Dropaut](https://pytorch.org/docs/master/generated/torch.nn.Dropout.html)

In [2]:
class CNNNet(nn.Module):

    def __init__(self, num_classes=2):
        super(CNNNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2), # двумерная свертка
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2), # пкллинг
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Linear(4096, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [6]:
cnnnet = CNNNet()

In [7]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device="cpu"):
    for epoch in range(epochs):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)
        
        model.eval()
        num_correct = 0 
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output,targets) 
            valid_loss += loss.data.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(output), dim=1)[1], targets).view(-1)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)

        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch, training_loss,
        valid_loss, num_correct / num_examples))

In [8]:
def check_image(path):
    try:
        im = Image.open(path)
        return True
    except:
        return False
img_transforms = transforms.Compose([
    transforms.Resize((64,64)),    
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225] )
    ])
train_data_path = "pytorch_learning/train/"
train_data = torchvision.datasets.ImageFolder(root=train_data_path,transform=img_transforms, is_valid_file=check_image)
val_data_path = "pytorch_learning/val/"
val_data = torchvision.datasets.ImageFolder(root=val_data_path,transform=img_transforms, is_valid_file=check_image)
batch_size=64
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,shuffle=True)
val_data_loader  = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=True)
if torch.cuda.is_available():
    device = torch.device("cuda") 
else:
    device = torch.device("cpu")

In [9]:
cnnnet.to(device)
optimizer = optim.Adam(cnnnet.parameters(), lr=0.001)

In [10]:
train(cnnnet, 
      optimizer,
      torch.nn.CrossEntropyLoss(), 
      train_data_loader,val_data_loader, 
      epochs=10, 
      device=device)

/home/konstantin/anaconda3/envs/pytorch-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch: 0, Training Loss: 0.85, Validation Loss: 0.69, accuracy = 0.66
Epoch: 1, Training Loss: 0.68, Validation Loss: 0.62, accuracy = 0.79
Epoch: 2, Training Loss: 0.59, Validation Loss: 0.73, accuracy = 0.53
Epoch: 3, Training Loss: 0.51, Validation Loss: 0.58, accuracy = 0.72
Epoch: 4, Training Loss: 0.47, Validation Loss: 0.44, accuracy = 0.76
Epoch: 5, Training Loss: 0.45, Validation Loss: 0.56, accuracy = 0.71
Epoch: 6, Training Loss: 0.42, Validation Loss: 0.36, accuracy = 0.85
Epoch: 7, Training Loss: 0.41, Validation Loss: 0.41, accuracy = 0.77
Epoch: 8, Training Loss: 0.37, Validation Loss: 0.32, accuracy = 0.86
Epoch: 9, Training Loss: 0.36, Validation Loss: 0.36, accuracy = 0.86


In [13]:
labels = ['cat','fish']
def predictor(image_for_predict):
    img = Image.open(image_for_predict) 
    img = img_transforms(img).to(device)

    prediction = F.softmax(cnnnet(img), dim=1)
    prediction = prediction.argmax()
    print(labels[prediction])

In [14]:
# cat
predictor("pytorch_learning/test/cat/2041806579_e4a7f31b32.jpg")

RuntimeError: Expected 4-dimensional input for 4-dimensional weight [64, 3, 11, 11], but got 3-dimensional input of size [3, 64, 64] instead

## Загрузка готовой сети из хаба

На самом деле AlexNet определен в самом pytorch и его вызов осуществляется в две строки (в данном случае мы вызываем предварительно натрененный алекснет)

Тут также доступны VGG, ResNet, Inception, DenseNet, SqueezeNet и т.д. 

[подробнее про доступные сети тут](https://pytorch.org/docs/stable/torchvision/models.html)

In [3]:
import torchvision.models as models
alexnet = models.alexnet(num_classes=1000, pretrained=True)

Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /home/konstantin/.cache/torch/checkpoints/alexnet-owt-4df8aa71.pth


In [5]:
print(alexnet)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 